![](NCSU-CSS.png)

# <font  color='red'>Week 13 - Homework (GWAS & GS)</font>

### Submit a separate page answering the six questions in this document

First, we're going to install packages and dependencies for plotting and modeling the genomic data
- GGPlot2 - https://ggplot2.tidyverse.org/, https://www.rdocumentation.org/packages/ggplot2/versions/3.3.2
- Sommer - https://www.rdocumentation.org/packages/sommer/versions/3.1

We're using an older version of Sommer since these are constantly updated (every 3 months) and the following code was developed for version 3.1

In [ ]:
install.packages('versions')
install.packages('ggplot2')
library(versions)
#available.versions("sommer")
install.versions("sommer", "3.1")
library(sommer)
library(ggplot2)

Read in the numericalized genotype data for the maize diversity panel within mdp_num_geno.csv.

In [ ]:
geno = read.csv("mdp_num_geno.csv")
geno[1:10,1:5] #show a slice of the data frame

Note that genotype scores are counts of minor allele at each marker.

You may also want information on the genomic position of each marker, you can read that in from another file called MarkerInfo.csv.

In [ ]:
MarkerInfo = read.csv("MarkerInfo.csv")
head(MarkerInfo)

Get the trait data (mean values for each line):

In [ ]:
pheno = read.csv("mdp_pheno.csv")
head(pheno)

Let’s look at population structure with principal components analysis of the genotype matrix. Unfortunately, we cannot do PC decomposition of a matrix with missing values, so we need to fill those missing values somehow. This requires imputation, which should be done with great care, but for our purposes we will just impute the major allele homozygote for all NA values:

In [ ]:
gmat = as.matrix(geno[,-1])
rownames(gmat) = geno[,1]
gmat[is.na(gmat)] = 0

#find some marker columns that are all zeroes and drop them
dropcols = which(colMeans(gmat) == 0)
gmat = gmat[,-dropcols]
str(gmat)

Create the Principal Components using the prcomp function

In [ ]:
pcs = prcomp(gmat, center = T, scale = T, retx = T)
str(pcs)

The returned object is a dictionary-like list with a bunch of useful things in it. The variance associated with each PC is reported as standard deviations in the list component sdev. Let’s use that to compute the percent variance associated with the first 5 PCs:

In [ ]:
PC_vars = pcs$sdev^2
total_var = sum(PC_vars)
percent_var = PC_vars/total_var*100
percent_var[1:5]

Let’s display the lines on the first two PCs of marker data. We can pull out the scores of the lines on the first two PCs from the first two columns of pcs$x. To make some sense of things, we will color the lines from a couple of distinct breeding groups to see if they follow the patterns expected based on pedigrees.

In [ ]:
PCs_lines = data.frame(pcs$x[,1:2])
PCs_lines$Line = geno$Line
PCs_lines$group = ifelse(substr(PCs_lines$Line,1,3) == "CML", "CML",
                         ifelse(PCs_lines$Line %in% c("B37", "B73", "B104","A632"), "Stiff Stalk",
                                ifelse(PCs_lines$Line %in% c("MO17", "OH43", "B97", "OH7B"), "Non-Stiff Stalk", "NA")))

pc_plot = ggplot(PCs_lines, aes(x = PC1, y = PC2)) +
  geom_point(aes(colour = group))

pc_plot

# Question 1

Is there a relationship between days to pollen shed (dpoll) and population structure? Regress dpoll on the first three PCs from the analysis above and see if any of the three PCs are significantly related to dpoll. Here is some code to get you started:

In [ ]:
PCs_pheno = merge(PCs_lines, pheno, by = "Line")
PCs_pheno$PC3 = pcs$x[,3]
head(PCs_pheno)

Here is how I would regress EarHT on the first two principal components:

(Hint: Do the same but with dpoll and the first three PCs)

In [ ]:
EHT_pc1 = lm(formula = EarHT ~ PC1 + PC2, data = PCs_pheno)
summary(EHT_pc1)

# Question 2

Compute LD r<sup>2</sup> values between all pairs of the following markers: [marker chrom pos_Agpv1] See GWAS lecture.
> #### zagl1.2 1 4835558 
> #### zagl1.6 1 4835658 
> #### PZA00236.7 3 189437413 
> #### PZA01228.2 3 189861328 
> #### PZA03351.1 10 89122261

Use the ‘imputed’ data set "gmat" to start with. Set any heterozygous calls to 0 (code below) for this question, so that you can directly estimate haplotype frequencies from the genotype frequencies (this is a nice property of homozygous lines). Show me how you did this by hand, or by using some functions in base R or other software. Try not to use special software packages that are explicitly designed to compute LD statistics.

Here is how to get a slice of the dataframe for just these markers with heterozygous calls ('1') changed to ('0') or homozygous calls for the major allele:

In [ ]:
selector = c('zagl1.2', 'zagl1.6','PZA00236.7','PZA01228.2',
'PZA03351.1')
geno_sub = gmat[, selector]
geno_sub[geno_sub == 1] = 0
head(geno_sub)

# Question 3

Test the same set of 5 markers selected in Question 2 for association with dpoll. For this question, do NOT do any correction for population structure. Just do an ANOVA or regression of the trait on the number of minor alleles carried by each line at. Which markers would you declare signficantly associated with flowering time at p = 0.001 threshold?

First, merge all of data (3 PCs, 5 selected markers and phenotypes) together into All_pheno.

In [ ]:
Line = rownames(geno_sub)
geno_sub = cbind(Line,geno_sub)
rownames(geno_sub) = 1:nrow(geno_sub)
All_pheno = merge(PCs_pheno, geno_sub, by="Line")
head(All_pheno)

Hint: Use the same formula type above but for the 5 selected markers only

In [ ]:
# Add code here

# Question 4

Test the same set of 5 markers again, but this time include the first three PCs from the marker data as covariates in the analysis. How do your results change?

Hint: Add all 8 into the same formula

In [ ]:
# Add code here

# Question 5

Create a realized genomic relationship matrix from the marker data. We’ll use the A.mat() function in R package ‘sommer’ to do this and to fit mixed models.

In [ ]:
K = A.mat(X = gmat)
rownames(K) = rownames(gmat)
print(K[1:5,1:5])
heatmap(K, symm = T)

Notice that inbred lines should have diagonal elements near 2.0, and unrelated pairs should have off-diagonal elements near 0. Values greater than 0 indicate closer relationships than expected by chance, less than 0 are more distant than by chance.

Now we can re-fit the association tests using a mixed model with K as the relationship matrix via mmer2() function in sommer. 

As a first example, here is how I would fit the K (kinship) matrix to model the variance-covariance of the lines, with no fixed effects in the model:

In [ ]:
rand_mod = mmer2(fixed = dpoll ~ 1, random = ~g(Line), rcov = ~units, G = list(Line = K), data = All_pheno, silent = T)
summary(rand_mod)

‘dpoll ~ 1’ means we are fitting just the intercept as a fixed effect in this model. In the next model, you will see how to fit a fixed model term.

We can estimate the heritability for dpoll in this experiment using the variance components estimated:

In [ ]:
Vg = rand_mod$var.comp$`g(Line)`
Verr = rand_mod$var.comp$units
Vp = Vg + Verr
h2 = Vg/Vp
print(paste("Vg:", Vg))
print(paste("Verr:", Verr))
print(paste("Vp:", Vp))
print(paste("Heritability:", h2))

In this example, here is how I would fit the first PC as a fixed effect and the lines as random with covariance structure proportional to the K (kinship) matrix:

In [ ]:
mlm_ex = mmer2(fixed = dpoll ~ PC1, random = ~g(Line), rcov = ~units, G = list(Line = K), data = All_pheno, silent = T)
summary(mlm_ex)

Now, fit the association tests for the 5 selected markers using the relationship matrix (K) to model the genetic backgrounds/population structure. Compare the marker effect estimates from this model to the two previous models, explain which estimates might be more reliable.

For convenience > zagl1.2, zagl1.6, PZA00236.7, PZA01228.2, PZA03351.1

In [ ]:
# Add code here

# Question 6

Finally, let’s fit a genomic selection model to these data and cross-validate it. We first partition the lines according to their PC1 scores to see if we can create a model using 80% of lines with lower values, how well does it predict values of the other 20% of lines?

In [ ]:
ordered_lines = PCs_lines[order(PCs_lines$PC1), "Line"]
last = length(ordered_lines)
first = ceiling(0.8*last)
print(paste("First and last indices of lines in test set:", first, ",", last))
test_set = ordered_lines[first:last]
head(test_set)

Make a new variable ‘dpoll_train’ and use the vector of test_set line names to set their dpoll values to missing.

In [ ]:
pheno$dpoll_train = pheno$dpoll
pheno[pheno$Line %in% test_set, "dpoll_train"] = NA

Now fit the genomic selection model on the training data and predict the values for all individuals

In [ ]:
gs_mod = mmer2(fixed = dpoll_train ~ 1, random = ~g(Line), rcov = ~units, G = list(Line = K), data = pheno, silent = T)
summary(gs_mod)

We can extract the predicted values from gs_mod$fitted.y

In [ ]:
pheno$dpoll_pred = gs_mod$fitted.y

Your job is to compare the predicted values to the observed dpoll values of only the lines in the test set. Compute the correlation, how good is the genomic prediction model compared to direct phenotypic measurements? Also, draw a scatterplot of the observed values on y-axis vs predicted values on x-axis for only the test set lines.